In [ ]:
import os


account_id = os.getenv["AWS_ACCOUNT"]
image_config = {'RepositoryAccessMode': 'Platform'}
primary_container = {
    'ContainerHostname': 'ModelContainer',
    'Image': f'{account_id}.dkr.ecr.us-west-2.amazonaws.com/whisper-gpu:latest',
    'ImageConfig': image_config,
    'ModelDataUrl': 's3://sagemaker-us-west-2-{account_id}/whisper-gpu-deploy/model/model.tar.gz'
}
model_name="whisper-gpu-model"
execution_role_arn = 'arn:aws:iam::{account_id}:role/SageMakerExecutionRole'

In [ ]:
import boto3
sm = boto3.client('sagemaker')

try:
    resp = sm.create_model(
        ModelName=model_name,
        PrimaryContainer=primary_container,
        ExecutionRoleArn=execution_role_arn
    )
except Exception as e:
    print(f'error calling CreateModel operation: {e}')
else:
    print(resp)

In [ ]:
endpoint_config_name = 'whisper-gpu-endpoint-config'
sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'WhisperGpu',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.p3.8xlarge'
        },
    ],
)

In [ ]:
endpoint_name = 'whisper-gpu-endpoint'
sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
sm.describe_endpoint(EndpointName=endpoint_name)